In [1]:
import pandas as pd
import random as rd
import numpy as np

In [2]:
def separarTreinoTeste(dfA):
    dfB = dfA
    dfC = pd.DataFrame()
    for i in range (len(dfA)):
        vetor = dfA.iloc[i, 0]
        if (rd.random() > 0.8): # vai p teste
            dfB = pd.concat([dfB, pd.DataFrame(data=[[vetor]], columns=['noticias'])], ignore_index=True)
        else:
            dfC = pd.concat([dfC, pd.DataFrame(data=[[vetor]], columns=['noticias'])], ignore_index=True)

    return dfB, dfC

In [3]:
def prepararDfTeste (dfA, dfB):
    # embaralha os dfs
    dfEmbaralhado = pd.concat([dfA, dfB], ignore_index=True)
    indices = list(dfEmbaralhado.index)
    rd.shuffle(indices)
    
    tags = []
    noticias = []

    # cria uma coluna pra tag
    for i in indices:
        news = dfEmbaralhado.iloc[i, 0]
        tags.append(news[0])
        noticias.append(news[1:])

    return pd.DataFrame({'tag': tags, 'noticia': noticias})

In [4]:
def prepararDfsTreino (dfF, dfR):
    # Falsas
    palavrasFalsas = []
    for i in range(len(dfF)):
        noticia = dfF.iloc[i,0][1:]
        palavrasFalsas.extend(noticia)
    dfF = pd.DataFrame({'palavras': palavrasFalsas})
    dfF = dfF.groupby('palavras').size().reset_index(name='aparicoesFalsas')

    palavrasFalsas = []
    for i in range(len(dfR)):
        noticia = dfR.iloc[i,0][1:]
        palavrasFalsas.extend(noticia)
    dfR = pd.DataFrame({'palavras': palavrasFalsas})
    dfR = dfR.groupby('palavras').size().reset_index(name='aparicoesReais')
    palavras = []

    return dfF, dfR

In [120]:
def evaluate(dfReal, dfFalso, noticiasTeste, limiar):
    # Junta os dataframes e une as palavras
    dfJunto = dfReal.merge(dfFalso, how='outer')

    # Conta quantas palavras tem em uma e não tem na outra
    nReal = dfJunto.loc[dfJunto['aparicoesReais'].isna(), 'palavras'].count()
    nFalsa = dfJunto.loc[dfJunto['aparicoesFalsas'].isna(), 'palavras'].count()

    # Preenche as palavras que não aparecem com 1
    dfJunto.fillna(1, inplace=True)

    # Soma as palavras com 0 aparições com 1 para evitar multiplicação por 0
    dfJunto.loc[dfJunto['aparicoesReais'] > 1, 'aparicoesReais'] += 1
    dfJunto.loc[dfJunto['aparicoesFalsas'] > 1, 'aparicoesFalsas'] += 1

    # Remove linhas onde a coluna 'palavras' é vazia ou só espaço
    dfJunto = dfJunto[dfJunto['palavras'].str.strip() != '']

    # Mantém só palavras que têm pelo menos uma letra (a–z ou A–Z ou acentuadas)
    dfJunto = dfJunto[dfJunto['palavras'].str.match(r'^[A-Za-zÀ-ÖØ-öø-ÿ]+$', na=False)]

    # Soma o total de aparições
    totalReal = dfJunto['aparicoesReais'].sum()
    totalFalso = dfJunto['aparicoesFalsas'].sum()

    # Balanceia a ordem de grandeza das aparições
    dfJunto['aparicoesFalsas'] *= totalReal / totalFalso

    # Faz o coeficiente de bray courtis
    dfJunto['bc'] = abs(dfJunto['aparicoesReais'] - dfJunto['aparicoesFalsas']) / (dfJunto['aparicoesReais'] + dfJunto['aparicoesFalsas'])

    # Limpa palavras com coeficiente menor que 0.3 (palavras com aparições equivalentes em ambos casos)
    dfJunto = dfJunto.loc[dfJunto['bc'] > limiar]

    # Pega o total de aparições após NLP
    tFalsas = dfJunto['aparicoesFalsas'].sum()
    tReais = dfJunto['aparicoesReais'].sum()

    # Define um df de teste
    dfJuntoTeste = dfJunto

    # Faz os calculos das probabilidades
    dfJuntoTeste['aparicoesFalsas'] = dfJunto['aparicoesFalsas'] / (nFalsa + tFalsas)
    dfJuntoTeste['aparicoesReais'] = dfJunto['aparicoesReais'] / (nReal + tReais)

    # Pega o log na base 10
    dfJuntoTeste['aparicoesFalsas'] = np.log10(dfJuntoTeste['aparicoesFalsas'])
    dfJuntoTeste['aparicoesReais'] = np.log10(dfJuntoTeste['aparicoesReais'])

    # NOTICIA:
    
    indices = ['fF', 'fR', 'rF', 'rR']
    dfDados = pd.DataFrame(0, index=indices, columns=['num'])

    for i in range (len(noticiasTeste)):
        noticia = noticiasTeste.iloc[i, 1]
        tag = noticiasTeste.iloc[i, 0]

        # Cria um df com a notícia limpa
        dfNoticia = pd.DataFrame(data = noticia, columns=['palavra'])

        # Junta o df da noticia com o das probabilidades
        dfNoticia = dfNoticia.merge(dfJuntoTeste, how='inner', left_on='palavra', right_on='palavras')

        # Faz a soma das probabilidades
        iFalsa = dfNoticia['aparicoesFalsas'].sum()
        iReal = dfNoticia['aparicoesReais'].sum()

        # Confere o resultado
        if (iFalsa > iReal): # Falsa
            resultado = 'F'
        elif (iReal == iFalsa): # Empate
            resultado = 'E'
        else:
            resultado = 'R'
            
        chave = tag + resultado
        try:
            dfDados.loc[chave, 'num'] += 1
        except:
            pass

    return dfDados

In [6]:
def calcularDPMedia(dados):
    n = len(dados)
    media = sum(dados) / n
    soma_dif_quadrados = sum((x - media) ** 2 for x in dados)
    dp = (soma_dif_quadrados / (n - 1)) ** 0.5
    return dp, media


In [7]:
def calcularDPMedia(dados):
    n = len(dados)
    media = sum(dados) / n

    somaDiferencaQuadrados = 0
    for valor in dados:
        somaDiferencaQuadrados += (valor - media) ** 2

    dp = (somaDiferencaQuadrados / (n-1)) ** 0.5
    return dp, media

In [8]:
def calcularIntervaloConfianca(dp, media, n, z=1.96):
    margem = z * dp / (n ** 0.5)
    return media - margem, media + margem

In [ ]:
#Calcula o Generalized Jaccard

#dfUnido['jaccard'] = np.minimum(dfUnido['aparicoesReais'], dfUnido['aparicoesFalsas']) / np.maximum(dfUnido['aparicoesReais'], dfUnido['aparicoesFalsas'])
#https://distancia.readthedocs.io/en/latest/GeneralizedJaccard.html

In [ ]:
dfFalsoOriginal = pd.read_json('../data/train/fakeTrain.json', orient='records', lines=True)
dfRealOriginal = pd.read_json('../data/train/realTrain.json', orient='records', lines=True)

matrizesDeConfusao = [[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]]

for i in range(5):
    print(i)
    dfFalso = dfFalsoOriginal.copy()
    dfReal = dfRealOriginal.copy()

    # separa em treino e teste
    [dfFalsoTeste, dfFalsoTreino] = separarTreinoTeste(dfFalso)
    [dfRealTeste, dfRealTreino] = separarTreinoTeste(dfReal)

    # mistura os dados de teste
    dfTestes = prepararDfTeste(dfFalsoTeste, dfRealTeste)

    # prepara os dados de treino (junta as palavras e conta as aparições)
    [dfFalsoTreino, dfRealTreino] = prepararDfsTreino(dfFalsoTreino, dfRealTreino)

    # avalia o modelo
    matrizesRounds = []
    for i in range(1, 10): 
        limiar = i/10
        print(limiar)
        matrizDeConfusao = evaluate(dfRealTreino, dfFalsoTreino, dfTestes, limiar)
        matrizesRounds.append(matrizDeConfusao)
    
    matrizesDeConfusao.append(matrizesRounds)

    # guarda a matriz de confusão

In [152]:
sum(matrizesDeConfusao[1][1]['num'])

3114

In [153]:
matrizesDeConfusao[1][1]

,num
fF,1437
fR,195
rF,0
rR,1482


In [155]:
acuracidade = round(float(matrizesDeConfusao[1][1].loc['rR', 'num'] + matrizesDeConfusao[1][1].loc['fF', 'num']) / sum(matrizesDeConfusao[1][1]['num']), 4)
acuracidade

0.9374

In [174]:
matrizesDeConfusao[0][8]

0.9

In [203]:
matrizesDeConfusao[0][1]

0.2

In [ ]:
for i in range(1, 6):
    print(matrizesDeConfusao[i][8])
    acuracidade = round(float(matrizesDeConfusao[1][i].loc['rR', 'num'] + matrizesDeConfusao[1][i].loc['fF', 'num']) / sum(matrizesDeConfusao[1][i]['num']), 4)

In [215]:
# matrizesDeConfusao[round][limiar]
mediasAcuracidades = []
mediaPrecisoesReais = []

acuracidades = []
sensisFalsas = []
sensisReais = []
precisaosFalsas = []
precisaosReais = []
f1Score = []

for i in range(0, 9):
    for j in range(1, len(matrizesDeConfusao)):
        acuracidade = round(float((matrizesDeConfusao[j][i].loc['rR', 'num'] + matrizesDeConfusao[j][i].loc['fF', 'num']) / sum(matrizesDeConfusao[j][i]['num'])), 4)
        acuracidades.append(acuracidade)

        #sensiFalsa =
        #sensisFalsas.append(sensisFalsas)
        
        precReal = round(float((matrizesDeConfusao[j][i].loc['rR', 'num']) / (matrizesDeConfusao[j][i].loc['rR', 'num'] + matrizesDeConfusao[j][i].loc['fR', 'num'])), 4)
        precisaosReais.append(precReal)

    mediaAcuracidades = round((sum(acuracidades) / len(acuracidades)), 4)
    mediasAcuracidades.append(mediaAcuracidades)

    mediaPrecisaoReais = round((sum(precisaosReais) / len(precisaosReais)), 4)
    mediaPrecisoesReais.append(mediaPrecisaoReais)


In [216]:
mediasAcuracidades

[0.9337, 0.9351, 0.9367, 0.9403, 0.9435, 0.9461, 0.9486, 0.9514, 0.9553]

In [217]:
mediaPrecisoesReais

[0.8776, 0.8801, 0.8826, 0.8887, 0.8942, 0.8987, 0.9033, 0.909, 0.9174]

In [ ]:
fF = [], fR = [], rF = [], rR = []
for i in range(len(matrizesDeConfusao)):
    fF.append(matrizesDeConfusao[i].loc['fF', 'num'])
    fR.append(matrizesDeConfusao[i].loc['fR', 'num'])
    rF.append(matrizesDeConfusao[i].loc['rF', 'num'])
    rR.append(matrizesDeConfusao[i].loc['rR','num'])
                                        

In [ ]:
dpPrecFalsas, mediaPrecFalsas = calcularDPMedia(precsFalsas)
minPrecisaoFalsas, maxPrecisaoFalsas = calcularIntervaloConfianca(dpPrecFalsas, mediaPrecFalsas, len(precsFalsas))

dpPrecReais, mediaPrecFReais= calcularDPMedia(precsReais)
minPrecisaoReais, maxPrecisaoReais = calcularIntervaloConfianca(dpPrecReais, mediaPrecFReais, len(precsReais))

dpSensiReais, mediaSensiReais= calcularDPMedia(sensisReais)
minSensiReais, maxSensiReais = calcularIntervaloConfianca(dpSensiReais, mediaSensiReais, len(sensisReais))

dpSensiFalsas, mediaSensiFalsas= calcularDPMedia(sensisFalsas)
minSensiFalsas, maxSensiFalsas = calcularIntervaloConfianca(dpSensiFalsas, mediaSensiFalsas, len(sensisFalsas))

dpAcuracidade, mediaAcuracidade= calcularDPMedia(acuracias)
minAcuracidade, maxAcuracidade = calcularIntervaloConfianca(dpAcuracidade, mediaAcuracidade, len(acuracias))


In [ ]:
dados = {
    'Métrica': ['Precisão Falsas', 'Precisão Reais', 'Sensibilidade Reais', 'Sensibilidade Falsas', 'Acuracidade'],
    'Mínimo': list([
        minPrecisaoFalsas,
        minPrecisaoReais,
        minSensiReais,
        minSensiFalsas,
        minAcuracidade
    ]),
    'Máximo': list([
        maxPrecisaoFalsas,
        maxPrecisaoReais,
        maxSensiReais,
        maxSensiFalsas,
        maxAcuracidade
    ])
}

dfIntervalos = pd.DataFrame(dados)

dfIntervalos['Mínimo'] = dfIntervalos['Mínimo'].round(4)
dfIntervalos['Máximo'] = dfIntervalos['Máximo'].round(4)

print(dfIntervalos)